##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Concrete functions

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/concrete_function">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/concrete_function.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/concrete_function.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/concrete_function.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>


In the guide to [AutoGraph and `tf.functions`](function.ipynb) you saw how to use `tf.function`. This guide dives into the details of: 

* `tf.function` Tracing
* `tf.function` Signatures
* The Concrete functions generated by tracing:
  * How to access them
  * How to use them

These details only become important:

* If you're experiencing performance issues due to undesired tracing
  of a `tf.funcion`.
* When you need precise control over the TensorFlow Graphs generated by
  `tf.function`. For example for exporting the model to
  [TensorFlow Lite](https://tensorflow.org/lite/) using
  `tf.lite.Converter.from_concrete_functions`.


## Background

In TensorFlow 2, eager execution is on by default. TensorFlow's eager
execution is an imperative programming environment that evaluates operations
immediately, without building graphs. Operations return values instead
of constructing a computational graph to run later. Here is a [detailed guide on eager
execution](eager.ipynb).

Running imperatively makes development and debugging
more interactive, but doesn't allow for easy exporting.

The `tf.function` API makes it possible to save models as graphs.

## Terminology

The following terminology is used in this document:

*   **Signature** - A description of the inputs and outputs for a set of operations.
* **Polymorphic function** - Python callable that encapsulates several
    concrete function graphs behind one API.  
* **Concrete function** - Graph with a single signature.
  


## Setup

In [0]:
import traceback
import textwrap

!pip install tf_nightly


In [0]:
import tensorflow as tf

## Create a `tf.function`

Annotating a function with `tf.function` generates a *polymorphic function*
containing those operations. All operations that are not annotated with
`tf.function` will be evaluated with eager execution. The examples below show a quick example of `tf.function` usage.

In [0]:
@tf.function
def square(x):
  return x*x

In [0]:
square(2).numpy()

Remember that the python decorator syntax just calls the decorator with the decorated object as input:

In [0]:
def pow(x,y):
  return x ** y

pow = tf.function(pow)

In [0]:
pow(3,4).numpy()

### Attach a `tf.function` method to a `tf.Module`

The `tf.function` can be optionally stored as part of a `tf.Module` object. The `tf.Module` class provides features for tracking variables and saving [checkpoints](checkpoints.ipynb) and [models](saved_model.ipynb).

Classes like `keras.layers.Layer` and `keras.Model` are subclasses of Module.

In [0]:
class Pow(tf.Module):
  def __init__(self, exponent):
    self.exponent = tf.Variable(exponent, dtype = tf.float32, name='Pow/exponent')

  @tf.function
  def __call__(self, x):
    return x ** self.exponent

In [0]:
pow = Pow(3)

In [0]:
pow.variables

In [0]:
pow(tf.constant(2.0)).numpy()

In [0]:
pow.exponent.assign(4)
pow(tf.constant(2.0)).numpy()

In [0]:
tf.saved_model.save(pow, 'pow')

In [0]:
reloaded_pow = tf.saved_model.load('pow')

In [0]:
reloaded_pow(tf.constant(3.0)).numpy()

### Assign a `tf.function` as an attribute
If you assign a `tf.Module` or a `tf.function` as an attribute of a module it will be serialized as well:

In [0]:
mod = tf.Module()
mod.increment_by = tf.Variable(2.0)

@tf.function
def increment(x):
  return x+mod.increment_by

mod.inc = increment
mod.inc(tf.constant(1.0)).numpy()

In [0]:
mod.cube = Pow(3)
mod.cube(tf.constant(2.0)).numpy()

In [0]:
mod.variables

In [0]:
tf.saved_model.save(mod, 'mod')
reloaded_mod = tf.saved_model.load('mod')

In [0]:
reloaded_mod.inc(4.0).numpy()

In [0]:
reloaded_mod.cube(4.0).numpy()

### Interoperability with `tf.keras`

Keras classes like `keras.Model` and `keras.layers.Layer` are fully compatible with `tf.function` and `tf.Module`.

For example, build a simple model:

In [0]:
linear = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])
linear.compile(optimizer='adam', loss='mean_squared_error')
linear.fit(x=[-1, 0, 1, 2, 3, 4], y=[-3, -1, 1, 3, 5, 7], epochs=50, verbose=0)

In [0]:
linear(tf.constant([[1],[2]]))

Inspect it's variables

In [0]:
linear.variables

Now attach it to a `tf.Module`:

In [0]:
module = tf.Module()
module.linear = linear

The `tf.Module` also tracks the `tf.Variable`s:

In [0]:
module.variables

The `tf.Module` will export the contents of the `keras.Model` as well:

In [0]:
tf.saved_model.save(module,'module')

In [0]:
reloaded = tf.saved_model.load('module')

In [0]:
reloaded.linear([[1.0]])

## Tracing

The objects returned from `tf.function` are polymorphic functions. They will accept python objects, or `tf.Tensors` with any shape or `tf.dtype` as input.

In the background TensorFlow builds `tf.Graph`s representing the calculation. 
This graph is wrapped in a python callable: a concrete function. Each concrete function can only handle a single input signature.

`tf.function` traces the python function each time in needs to create a concrete function. The easiest way to see when a function is traced is to add a call to print: 


In [0]:
@tf.function
def mul(a, b):
  print('Tracing:\n    {a}\n    {b}\n'.format(a=a, b=b))
  return a*b

### Dtypes and shapes

If you call the polymorphic function with two different types of input, it will trace once for each:

In [0]:
# Trace with ints
mul(tf.constant(2), tf.constant(3)).numpy()

In [0]:
# Trace with floats
mul(tf.constant(2.0), tf.constant(3.0)).numpy()

When you call it again with the same input types, it dispatches to an existing function instead of tracing:

In [0]:
# Call with ints again => no trace
mul(tf.constant(10), tf.constant(10))

Changing the sizes of the inputs also triggers a trace (setting `tf.function(experimental_relax_shapes=True)` may reduce this): 

In [0]:
# Trace with vectors
mul(tf.constant([1.0,3.0]), tf.constant(3.0)).numpy()

In [0]:
# Trace with different-sized vectors
mul(tf.constant([1.0,2.0,3.0, 4.0]), tf.constant(3.0))

### Immutable python objects

If you pass an immutable python object, like a `int`, `str`, or `tuple` to a `tf.function`, it executes a trace for each *value* of those python objects.

This is useful to control what gets included in the `tf.Graph` (See: [The Autograph Guide](function.ipynb) for more details).


In [0]:
@tf.function
def mul(a, b):
  print('Tracing:\n    {a}\n    {b}\n'.format(a=a, b=b))
  return a*b

In [0]:
# Trace for a=3.0
mul(3.0, tf.constant(3.0)).numpy()

In [0]:
# Don't trace for a=3.0 the second time:
mul(3.0, tf.constant(3.0)).numpy()

Caution: It is easy to cause many traces by passing unique python values. This can be a significant performance problem. Often passing a `tf.Tensor` value is the solution.

This loop traces the function for each unique int:

In [0]:
@tf.function
def power(a,b):
  print('Tracing "power": a={}'.format(a))
  return a**b

In [0]:
p = tf.constant(2)
for n in range(12):
  power(n,p)

On the second run each int has been traced, so there's no tracing to do:

In [0]:
p = tf.constant(2)
for n in range(12):
  power(n,p)

To avoid excess retracing be sure to pass a `tf.Tensor` instead of python numbers or strings:

In [0]:
p = tf.constant(2)
for n in tf.range(12):
  power(n,p)

To shut off tracing altogether, pass a signature to the `tf.function` decorator:

In [0]:
@tf.function(input_signature=(
    tf.TensorSpec(shape=[], dtype=tf.float32),
    tf.TensorSpec(shape=[], dtype=tf.float32),)
)
def power_with_sig(a,b):
  print('Tracing "power_with_sig"')
  return a**b

In [0]:
power_with_sig(3.0, 3.0).numpy()

In [0]:
try:
  power_with_sig(tf.constant([1.0,2.0,3.0]),tf.constant(3.0))
  assert False
except ValueError:
  traceback.print_exc(limit=1)

### Example: Dropout

Retracing for specific values gives you control over what code gets generated by the `tf.function`.


In [0]:
class Dropout(tf.Module):
  def __init__(self, rate, name=None):
    super(Dropout, self).__init__(name)
    self.rate = tf.Variable(rate, dtype = tf.float32, trainable=False)

  @tf.function
  def __call__(self, x, training=True):
    print(textwrap.dedent("""
                          Tracing "Dropout":
                              training = {}
                              x = {}
                              name = {:s}
                          """.format(training, x, self.name)))
    if training:
      print('    - Train branch\n')
      mask = tf.random.uniform(x.shape) > self.rate
      return x * tf.cast(mask, tf.float32)/self.rate
    else:
      print('    - Test branch\n')
      return x

Create an instance of this simple `Dropout` layer:

In [0]:
dropout = Dropout(0.5)

The first time you call it with a python `training=True` as input, it traces the `training` branch:

In [0]:
dropout(tf.range(10, dtype=tf.float32), training=True).numpy()

The second time, it doesn't need to re-trace the branch:

In [0]:
dropout(tf.range(10, dtype=tf.float32), training=True).numpy()

Passing `training=False` triggers a trace on the first run since this is a different python value:

In [0]:
dropout(tf.range(10, dtype=tf.float32), training=False).numpy()

In [0]:
dropout(tf.range(10, dtype=tf.float32), training=False).numpy()

If you pass a `bool` tensor, it uses TensorFlow autograph rewrite the `if` to a `tf.cond`m and traces both branches:

In [0]:
dropout(tf.range(10, dtype=tf.float32), training=tf.constant(False)).numpy()

This captures the control flow in a single concrete function.

In [0]:
 dropout(tf.range(10, dtype=tf.float32), training=tf.constant(True)).numpy()

In [0]:
dropout(tf.range(10, dtype=tf.float32), training=tf.constant(False)).numpy()

### Other python objects

Since the generated `tf.Graphs` cannot contain complex python objects, these are included by tracing and variable capture. 

The `tf.function` runs a separate trace for each **instance**. So each trace includes its own variables, and can set its behavior based on the instance.

The most common usage is on methods of Module, Layer or Module:

In [0]:
dropout_a = Dropout(0.5, name='dropout_a')

In [0]:
print(dropout_a(tf.range(10, dtype=tf.float32), True).numpy())
print(dropout_a(tf.range(10, dtype=tf.float32), True).numpy())

In [0]:
dropout_b = Dropout(0.5, name='dropout_b')

In [0]:
print(dropout_b(tf.range(10, dtype=tf.float32), True).numpy())
print(dropout_b(tf.range(10, dtype=tf.float32), True).numpy())

But the behavior is the same on a stand-alone `tf.function`.

In [0]:
@tf.function
def run(callable, x):
  print('Tracing "run":\n    callable = {}\n    x = {}\n'.format(callable, x))
  return callable(x)

In [0]:
def plus_1(x):
  return x+1

print(run(plus_1, tf.constant(2.0)).numpy())
print(run(plus_1, tf.constant(5.0)).numpy())

The tracing one `tf.function` can trigger tracing in another:

In [0]:
print(run(dropout, tf.range(10.0)).numpy())
print(run(dropout, tf.range(10.0)).numpy())

### Weak references

Caution: Each trace only keeps a [weak-reference](https://docs.python.org/3/library/weakref.html) to any `tf.Variable`. If the variable is not kept alive by another reference, the trace may become unusable.

For example here's a `tf.function` that refers to `var` from the enclosing scope:

In [0]:
@tf.function
def plus_var(x):
  print('Tracing "plus_var":\n    x = {}\n    var = {}\n\n'.format(x, var.name))
  return x + var

Trace the function with one variable:

In [0]:
var = tf.Variable(1, name="IntVar")
plus_var(tf.constant([1,2])).numpy()

And with another variable:

In [0]:
var = tf.Variable(2.0, name="FloatVar")
plus_var(tf.constant([2.0, 10.0])).numpy()

That worked, but because you no longer have a reference to `"IntVar"`, that first trace is broken:

In [0]:
try:
  plus_var(tf.constant([1,2])).numpy()
  assert False
except tf.errors.FailedPreconditionError:
  traceback.print_exc(limit=1)

## Accessing concrete function

In the previous section you saw the conditions for triggering a new trace of a polymorphic `tf.function`. Each trace generates a new concrete function.

When you save `tf.Module` as a `tf.saved_model` It's those concrete functions that define the `tf.Graph`s that are exported. You don't save a `tf.function` you save the concrete functions that are created by tracing. 

To get a concrete function from the polymorphic `tf.function` you need to define the signature. Either:

*   Pass an `input_signature` to `tf.function`, and call the 
    `get_concrete_function()` method.
*   Pass a list of `tf.TensorSpec`s to `get_concrete_function`: `tf.TensorSpec(shape=[1], dtype=tf.float32)`.
*   Pass an example tensor of the correct shape and type to
    `get_concrete_function`: `tf.constant(1., shape=[1])`.

The following example shows how to define the `input_signature` parameter for
`tf.function`.

#### Using `input_signature`

Specify input tensors in the call to `tf.function` as shown below.
This `tf.function`can only execute on tensors that match the specified signatutre.

A `None` in the `shape` acts a wildcard. So this these `tf.TensroSpec` say "A float32 vector of any length".

This pattern can be very important if your `tf.function` is expected to handle sequences of different length, or images of different sizes for each batch (See [Transformer](../tutorials/text/transformer.ipynb) and [Deep Dream](../tutorials/generative/deepdream.ipynb) tutrorials for example).

In [0]:
@tf.function(input_signature=(
    tf.TensorSpec(shape=[None], dtype=tf.float32),
    tf.TensorSpec(shape=[None], dtype=tf.float32),)
)
def power_with_sig(a,b):
  print('Tracing "power_with_sig"\n')
  return a**b

Calling `get_concrete_function` will execute the trace (if necessary), and return a concrete function.

In [0]:
p = power_with_sig.get_concrete_function()
type(p)

In [0]:
p(tf.constant([2.0,3.0,4.0]), tf.constant([5.0,4.0,3.0])).numpy()

### Using `get_concrete_function`

In [0]:
@tf.function
def power(a,b):
  print('Tracing "power"\n')
  return a**b

In [0]:
float_power = power.get_concrete_function(
  a = tf.TensorSpec(shape=[], dtype=tf.float32),
  b = tf.TensorSpec(shape=[], dtype=tf.float32))

In [0]:
float_power(tf.constant(3.0),tf.constant(3.0))

Remember that you can also pass tensors to `get_concrete_function`, in that case it returns the concrete function that would run for those inputs:

In [0]:
row = tf.range(10)
col = tf.constant([[1],[2],[3]])

concrete_power = power.get_concrete_function(a = row, b = col)
concrete_power(row, col).numpy()

## Using a concrete function

A concrete function only accepts tensors as input:

In [0]:
float_power(tf.constant(2.0), tf.constant(3.0)).numpy()

In [0]:
try:
  float_power(2.0,3.0)
  assert False
except (ValueError, TypeError):
  traceback.print_exc(limit=1)

It also only accepts inputs of the correct dtype:

In [0]:
try:
  float_power(tf.constant(1),tf.constant(3))
  assert False
except tf.errors.InvalidArgumentError:
  traceback.print_exc(limit=1)

But it will try to execute even if the input tensors do not match the expected shape:

In [0]:
float_power(tf.constant([1.,2.,3.,4.,5.]),tf.constant(3.)).numpy()

In [0]:
try:
  float_power(tf.constant([1.,2.,3.]),tf.constant([4., 5.])).numpy()
  assert False
except tf.errors.InvalidArgumentError:  
  traceback.print_exc(limit=1)

By inspecting the concrete function you can see its inputs and outputs:

In [0]:
print(float_power.structured_input_signature)
print(float_power.structured_outputs)

## Python Objects in signatures

As you saw when tracing, each python object generates a new trace. Concrete functions represent a single `tf.Graph`, they don't do any retracing. When you call `get_concrete_function` with a python object as one of the arguments the object is **bound** to the function.

In [0]:
cube = power.get_concrete_function(
    a = tf.TensorSpec([], dtype=tf.float32),
    b = 3.0)

This `cube` function no longer has a `b` argument:

In [0]:
print(cube.structured_input_signature)

In [0]:
cube(tf.constant(10.0)).numpy()

This is very similar to the way that standard python classes bind methods, and applies equally when you run `get_concrete_function` from a method:

In [0]:
class Greeter(object):
  def __init__(self, greeting):
    self.greeting = greeting

  def greet(self, who):
    return " ".join([self.greeting, who])

p = Greeter("Hello")
m = p.greet
print(m)

In [0]:
print(m("TensorFlow!"))

When you have a `tf.function` decorating a method, similar rules apply:

In [0]:
class MyModel(tf.Module):
  def __init__(self, ins, outs):
    initializer = tf.initializers.GlorotNormal()
    self.W = tf.Variable(initializer([ins, outs]))
    self.B = tf.Variable(tf.zeros([outs], dtype = tf.float32))

  @tf.function
  def run(self, x):
    print('Tracing "MyModule":\n    x={}\n'.format(x))
    return tf.matmul(x, self.W)+self.B

In [0]:
mod = MyModel(ins=5, outs=3)

In [0]:
mod.run([[1.0,1.0,1.0, 1.0, 1.0]]).numpy()

If you call the method's `.get_concrete_function`, the `self` is automatically bound as the first argument:

In [0]:
concrete_run = mod.run.get_concrete_function(x = tf.TensorSpec([None, None]))

In [0]:
concrete_run(tf.constant([[1.0,1.0,1.0, 1.0, 1.0],
                          [2.0,2.0,2.0, 2.0, 2.0]])).numpy()

See how `self` is no longer part of the input signature:

In [0]:
print(concrete_run.structured_input_signature)
print(concrete_run.structured_outputs)

## Changes for TensorFlow 2.3

The following changes are currently available in TensorFlow nightly, and will be available in TensorFlow 2.3.

### Inspecting concrete function signatures

Printing a `ConcreteFunction` displays a summary of its input arguments (with types) and its output type:

In [0]:
print(float_power)

For polymorphic functions, the `pretty_printed_concrete_signatures()` method can be used to display a list of all signatures that have been traced so far:

In [0]:
print(power.pretty_printed_concrete_signatures())

### Bound arguments

As discussed above, when a concrete function is created, any arguments that are set to non-Tensor values are *bound* to those values.  Prior to TensorFlow 2.3, these bound arguments were simply removed from the concrete function's signature.  Starting with TensorFlow 2.3, they remain present in the signature, but have their default value set to the bound value.  For example, the argument `b` has its default value set to `2` when we create the following concrete function:

In [0]:
square = power.get_concrete_function(a=tf.TensorSpec(None, tf.float32), b=2)
print(square)

When calling `square`, you may either omit `b`, or explicitly set it to the value that was used during tracing:

In [0]:
print(square(tf.constant(5.0)))  # ok: use default value
print(square(tf.constant(5.0), b=2))  # ok: explicit value == bound value

Setting a bound argument to any value other than its default is an error:

In [0]:
try:
  print(square(tf.constant(10.0), b=4.0).numpy())  # error: explicit value != bound value
  assert False
except TypeError:
  traceback.print_exc(limit=1)

### CompositeTensor arguments

Starting with TensorFlow 2.3, you can define concrete functions that accept `CompositeTensor`s (such as `tf.RaggedTensor`, `tf.Dataset` iterators, and `tf.SparseTensor`).  E.g.:

In [0]:
ragged_power = power.get_concrete_function(
    a = tf.RaggedTensorSpec([None, None], dtype=tf.float32),
    b = tf.TensorSpec([], dtype=tf.float32)
)
print(ragged_power(tf.ragged.constant([[1.0, 2.0], [3.0]]),
                   tf.constant(2.0)))

### Nested arguments

Starting with TensorFlow 2.3, you can define concrete functions that accept nested `dict`s, `list`s, `tuple`s, `namedtuple`s, and [`attr`](https://www.attrs.org/en/stable/)s of tensors (or composite tensors).  E.g.:

In [0]:
@tf.function
def sum_features(features):
  assert isinstance(features, dict)
  return sum(t for t in features.values())

features = {
    'a': tf.constant([[1.0], [2.0], [8.0]]),
    'b': tf.constant([[5.0], [6.0], [0.0]]),
    'c': tf.ragged.constant([[10.0], [20.0, 30.0, 40.0], [50.0]])
}
print(sum_features(features))

concrete_sum_features = sum_features.get_concrete_function(features)
print(concrete_sum_features(features))

The value passed to a concrete function must have the same nesting structure that was used when the concrete function was traced.  If you pass in a different structure, then a `TypeError` is raised, with information about the structure that was accepted and the structure that was used:

In [0]:
try:
  concrete_sum_features({'a': tf.constant([[1.0]])})
  assert False
except TypeError:
  traceback.print_exc(limit=1)